In [ ]:
# Graphing function definitions
import os
import json
import time
import numpy as np
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

# Data loader routine into wide form
# Date | Stock | Reality | Prediction | Delta | RMSE....
def transform_data(stock, date, reality, prediction):
    data = []
    reality_series = pd.Series(reality)
    reality_diff_series = reality_series.diff().shift(-1)
    prediction_series = pd.Series(prediction)
    prediction_diff_series = prediction_series.diff().shift(-1)
    for i in range(1,len(date)):
        reality_series_head = reality_series.head(i)
        reality_diff_series_head = reality_diff_series.head(i)
        prediction_series_head = prediction_series.head(i)
        prediction_diff_series_head = prediction_diff_series.head(i)
    
        item = {"Date": date[i], "Stock": stock, "Prediction": prediction[i], "Reality": reality[i]}
        item["Price Delta - Predicted"] = prediction[i-1] - prediction[i]
        item["Price Delta - Reality"] = reality[i-1] - reality[i]
        item["Price Delta (%) - Predicted"] =  100*(prediction[i-1] - prediction[i])/prediction[i-1]
        item["Price Delta (%) - Reality"] = 100*(reality[i-1] - reality[i])/reality[i-1]
        item["Prediction Error"] = prediction[i] - reality[i]
        item["Prediction Error (Abs)"] = abs(prediction[i] - reality[i])
        item["Prediction Error (Cummulative)"] = sum([prediction[j] - reality[j] for j in range(i)])
        item["Prediction Error (Abs Cummulative)"] = sum([abs(prediction[j] - reality[j]) for j in range(i)])
        item["Correlation (rolling)"] = reality_series_head.corr(prediction_series_head) if i > 2 else None
        item["Correlation (rolling) - of Deltas"] = reality_diff_series_head.corr(prediction_diff_series_head) if i > 2 else None
        item["MSE"] = ((prediction_series_head - reality_series_head) ** 2).mean()
        item["RMSE"] = item["MSE"] ** 0.5
        item["RMSLE"] = ((np.log10(1 + prediction_series_head) - np.log10(1 + reality_series_head)) ** 2).mean()
        mse_model = ((prediction_series_head - reality_series_head) ** 2).mean()
        mse_baseline = ((reality_series_head.mean() - reality_series_head) ** 2).mean()
        item["R-Squared"] = 1 - mse_model / mse_baseline if i > 2 else None
        
        data.append(item)
    return data

def make_plot(wide_df):
    melted_df = df.copy().melt(["Date", "Stock"], var_name = "Metric", value_name = "Value")
    melted_df["color_id"] = melted_df["Stock"] + " - " + melted_df["Metric"]
    chart_base = alt.Chart(melted_df)
    
    selector_class = alt.selection_point(fields=['Metric'])
    chart_selector_class = chart_base.mark_bar().encode(
            y = f"Metric:N",
            color = alt.condition(selector_class, alt.value("red"), alt.value("lightgray"))
        ).add_params(selector_class)
    
    selector_stock = alt.selection_point(fields=['Stock'])
    chart_selector_stock = chart_base.mark_bar().encode(
            x = f"Stock:N",
            color = alt.condition(selector_stock, alt.value("red"), alt.value("lightgray"))
        ).add_params(selector_stock)
    
    chart_plot = chart_base.mark_line().encode(
            x = alt.X("Date"),
            y = alt.Y("Value").scale(zero=False),
            color = alt.Color("color_id", legend = None),
            tooltip = ["Stock", "Metric", "Date", "Value"]
        ).transform_filter(selector_class).transform_filter(selector_stock).properties(width = 2048).interactive()
    
    chart_box = chart_base.mark_boxplot().encode(
            x = alt.X("Stock"),
            column = alt.Column("Metric"),
            y = alt.Y("Value").scale(zero=False),
            color = alt.Color("color_id", legend = None),
        ).transform_filter(selector_class).transform_filter(selector_stock)
    
    return (chart_selector_stock & (chart_selector_class | chart_plot) & chart_box).configure(background='#BBBBBB')
    

In [ ]:
# Load Data
start = time.time()
data = []
for filename in os.listdir("../datasets/jse"):
    if filename.endswith("predicted"):
        path = os.path.join("../datasets/jse", filename)
        with open(path, "r") as infile:
            loaded = json.load(infile)
            data.extend(transform_data(filename[:5], loaded["Date"], loaded["Reality"], loaded["Prediction"]))
df = pd.DataFrame(data)
print(time.time() - start)

In [ ]:
make_plot(df)